

```
# Running HippoRAG on Colab
```



In [1]:
#https://stackoverflow.com/questions/76837647/how-to-downgrade-python-version-to-3-9-in-colab-without-creating-virtual-environ
#https://github.com/OSU-NLP-Group/HippoRAG/issues/3
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:11
🔁 Restarting kernel...


In [1]:
!conda create --name hippoEnv python=3.9

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / - done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local/envs/hippoEnv

  added / updated specs:
    - python=3.9


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bzip2-1.0.8                |       h4bc722e_7         247 KB  conda-forge
    ca-certificates-2024.7.4   |       hbcca054_0         151 KB  conda-forge
    ld_impl_linux-64-2.40      |       hf3520f5_7         691 KB  conda-forge
    libgcc-ng-14.1.0           |       h77fa898_0         822 KB  conda-forge
    libgomp-14.1.0             |       h77fa898_0         446 KB  conda-forge
    libsqlite-3.46.0           |       hde9e2c9_0         845 KB  conda-forge
    

In [2]:
%%shell
eval "$(conda shell.bash hook)"
conda activate hippoEnv

In [3]:
!python3 --version
# python is still 3.10 but now I can pip install requeriments.txt without crash

Python 3.10.13


In [4]:
!git clone https://github.com/OSU-NLP-Group/HippoRAG.git

Cloning into 'HippoRAG'...
remote: Enumerating objects: 440, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 440 (delta 170), reused 134 (delta 134), pack-reused 244
Receiving objects: 100% (440/440), 68.26 MiB | 20.62 MiB/s, done.
Resolving deltas: 100% (240/240), done.
Updating files: 100% (114/114), done.


In [5]:
# older versions of huggingface cuase the following error: 'split_torch_state_dict_into_shards'
!sed -i 's/huggingface-hub==0.20.3/huggingface-hub==0.23.4/' /content/HippoRAG/requirements.txt



In [6]:
# remove this packages from requirements.txt to avoid colab crash
!grep -Ev '^(ipython|ipykernel|tornado|prompt-toolkit|pyzmq)' /content/HippoRAG/requirements.txt > /content/HippoRAG/new_requirements.txt
!pip install -r /content/HippoRAG/new_requirements.txt

In [7]:
# to use Colbert uncomment
!cd /content/HippoRAG/exp; wget https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
!cd /content/HippoRAG/exp; tar -xvzf colbertv2.0.tar.gz


--2024-07-14 21:55:53--  https://downloads.cs.stanford.edu/nlp/data/colbert/colbertv2/colbertv2.0.tar.gz
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405924985 (387M) [application/octet-stream]
Saving to: ‘colbertv2.0.tar.gz’

colbertv2.0.tar.gz  100%[===================>] 387.12M  5.16MB/s    in 73s     

2024-07-14 21:57:07 (5.31 MB/s) - ‘colbertv2.0.tar.gz’ saved [405924985/405924985]

colbertv2.0/
colbertv2.0/artifact.metadata
colbertv2.0/vocab.txt
colbertv2.0/tokenizer.json
colbertv2.0/special_tokens_map.json
colbertv2.0/tokenizer_config.json
colbertv2.0/config.json
colbertv2.0/pytorch_model.bin


In [8]:
# create a new secret on colab to set OPENAI_API_KEY
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["TOGETHER_API_KEY"] = userdata.get('TOGETHER_API_KEY')


In [ ]:
# If you want langsmith logging uncomment

# from uuid import uuid4
# unique_id = uuid4().hex[0:8]
# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_PROJECT"] = f"HippoRagLoad - {unique_id}"
# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "LANGCHAIN_API_KEY"  # Update to your API key

In [9]:
# fix create_graph.py nltk download path
!if [ "$(head -n 1 /content/HippoRAG/src/create_graph.py)" != "import nltk" ]; then sed -i '1i\nltk.download("stopwords", download_dir="/tmp")' /content/HippoRAG/src/create_graph.py; sed -i '1i\nltk.data.path.append("/tmp/")' /content/HippoRAG/src/create_graph.py; sed -i '1i\import nltk' /content/HippoRAG/src/create_graph.py; fi



In [11]:
# Will use the corpus "case_study_movies" in the data folder with only 1 passages and Contriever

%env data=inventures2024
%env HF_RETRIEVER=colbertv2
# %env LLM_MODEL=openchat/openchat-3.5-1210
%env LLM_MODEL=gpt-3.5-turbo-1106
%env SYNONYM_THRESH=0.8
# GPUS=0,1,2,3
%env GPUS=0
# LLM API provider e.g., 'openai', 'together', see 'src/langchain_util.py'
%env LLM_API=openai
%env extraction_type=ner
# 'all' or 'the number of passages to use'
%env num_passages=4

# bash src/setup_hipporag.sh $DATA $HF_RETRIEVER $LLM $GPUS $SYNONYM_THRESH $LLM_API
# here are the commands inside setup_hipporag.sh, will run here to pass num_passages to 3

!echo ***Extracting Open Information***
# Running Open Information Extraction
!cd /content/HippoRAG; python src/openie_with_retrieval_option_parallel.py --dataset $data --llm $LLM_API --model_name $LLM_MODEL --run_ner --num_passages $num_passages # NER and OpenIE for passages
# !echo ***Extracting entities from ***
# !cd /content/HippoRAG; python src/named_entity_extraction_parallel.py --num_processes 1 --dataset $data --llm $LLM_API --model_name $LLM_MODEL  # NER for queries

!echo ***Creating ColBERT Graph***
# Creating ColBERT Graph
!cd /content/HippoRAG; python src/create_graph.py --dataset $data --model_name colbertv2 --extraction_model $LLM_MODEL --threshold $SYNONYM_THRESH --extraction_type $extraction_type --cosine_sim_edges


# Getting Nearest Neighbor Files
%env CUDA_VISIBLE_DEVICES=0
!echo ***Creating nearest neighbor file output/kb_to_kb.tsv ***
!cd /content/HippoRAG; python src/colbertv2_knn.py --filename output/kb_to_kb.tsv
!echo ***Creating nearest neighbor file output/query_to_kb.tsv***
!cd /content/HippoRAG; python src/colbertv2_knn.py --filename output/query_to_kb.tsv


!echo ***Creating graph for the second time...
!cd /content/HippoRAG; python src/create_graph.py --dataset $data --model_name colbertv2 --extraction_model $LLM_MODEL --threshold $SYNONYM_THRESH --create_graph --extraction_type $extraction_type --cosine_sim_edges


!echo ***Creating colbert indexing files...
!cd /content/HippoRAG; python3 src/colbertv2_indexing.py --dataset $data --phrase output/$data'_facts_and_sim_graph_phrase_dict_ents_only_lower_preprocess_ner.v3.subset.p' --corpus 'data/'$data'_corpus.json'


env: data=inventures2024
env: HF_RETRIEVER=colbertv2
env: LLM_MODEL=gpt-3.5-turbo-1106
env: SYNONYM_THRESH=0.8
env: GPUS=0
env: LLM_API=openai
env: extraction_type=ner
env: num_passages=4
***Extracting Open Information***
ner_gpt-3.5-turbo-1106_4
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100% 1/1 [00:02<00:00,  2.57s/it]
100% 1/1 [00:02<00:00,  2.71s/it]
100% 1/1 [00:03<00:00,  3.02s/it]
100% 1/1 [00:05<00:00,  5.57s/it]
/usr/local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
OpenIE saved to output/openie_inventures2024_results_ner_gpt-3.5-turbo-1106_4.json
***Creating ColBERT Graph***
[nltk_data] Downloading package stopwords to /tmp...
[nltk_data]   

Hi


In [12]:
# create the test program
%%shell
cat > /content/HippoRAG/src/hippo_test.py<< EOF

import sys
sys.path.insert(0,'/content/HippoRAG')
import argparse
from langchain_util import LangChainModel
from qa.qa_reader import qa_read
from hipporag import HippoRAG

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str)
    parser.add_argument('--query', type=str)
    args = parser.parse_args()

    hipporag = HippoRAG(corpus_name=args.dataset, extraction_model='openai', extraction_model_name='gpt-3.5-turbo-1106',
                 graph_creating_retriever_name='colbertv2', qa_model=LangChainModel('openai', 'gpt-3.5-turbo'))

    qa_few_shot_samples = None
    queries = [args.query]
    for query in queries:
        ranks, scores, logs = hipporag.rank_docs(query, top_k=2)
        retrieved_passages = [hipporag.get_passage_by_idx(rank) for rank in ranks]
        response = qa_read(query, retrieved_passages, qa_few_shot_samples, hipporag.qa_model)
        print(f"{response=}")
        print(ranks)
        print(scores)
        print(logs)
EOF

In [13]:
!cd /content/HippoRAG; python src/hippo_test.py --dataset $data --query "What is Macleod Foyer?"

Building Graph: 100% 99/99 [00:00<00:00, 314811.29it/s]
2024-07-14 22:03:34,628 - hipporag - INFO - Graph built: num vertices: 50, num_edges: 99


[Jul 14, 22:03:34] #> Creating directory data/lm_vectors/colbert/inventures2024/phrase/indexes/nbits_2 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "qu

In [ ]:
# !cd /content/HippoRAG; python src/hippo_test.py --dataset $data --query "Who directed the movie Black Hawk Down?"

!cd /content/HippoRAG; python src/hippo_test.py --dataset $data --query "Who is Laura Kilcrease?"


Building Graph: 100% 82/82 [00:00<00:00, 320533.95it/s]
2024-07-13 18:13:34,532 - hipporag - INFO - Graph built: num vertices: 43, num_edges: 82


[Jul 13, 18:13:34] #> Creating directory data/lm_vectors/colbert/inventures2024/phrase/indexes/nbits_2 


#> Starting...
nranks = 1 	 num_gpus = 1 	 device=0
{
    "query_token_id": "[unused0]",
    "doc_token_id": "[unused1]",
    "query_token": "[Q]",
    "doc_token": "[D]",
    "ncells": null,
    "centroid_score_threshold": null,
    "ndocs": null,
    "load_index_with_mmap": false,
    "index_path": null,
    "index_bsize": 64,
    "nbits": 2,
    "kmeans_niters": 20,
    "resume": false,
    "similarity": "cosine",
    "bsize": 64,
    "accumsteps": 1,
    "lr": 1e-5,
    "maxsteps": 400000,
    "save_every": null,
    "warmup": 20000,
    "warmup_bert": null,
    "relu": false,
    "nway": 64,
    "use_ib_negatives": true,
    "reranker": false,
    "distillation_alpha": 1.0,
    "ignore_scores": false,
    "model_name": null,
    "qu

In [ ]:
%cd /content/HippoRAG

/content/HippoRAG


In [ ]:
# # Will use the corpus "case_study_movies" in the data folder with only 3 passages and Contriever

# %env DATA=case_study_movies
# %env HF_RETRIEVER=facebook/contriever
# %env LLM_MODEL=gpt-3.5-turbo-1106
# %env SYNONYM_THRESH=0.8
# # GPUS=0,1,2,3
# %env GPUS=0
# # LLM API provider e.g., 'openai', 'together', see 'src/langchain_util.py'
# %env LLM_API=openai
# %env extraction_type=ner
# # 'all' or 'the number of passages to use'
# %env num_passages=3

# # bash src/setup_hipporag.sh $DATA $HF_RETRIEVER $LLM $GPUS $SYNONYM_THRESH $LLM_API
# # here are the commands inside setup_hipporag.sh, will run here to pass num_passages to 3

# # Running Open Information Extraction
# !cd /content/HippoRAG; python src/openie_with_retrieval_option_parallel.py --dataset $DATA --llm $LLM_API --model_name $LLM_MODEL --run_ner --num_passages $num_passages # NER and OpenIE for passages
# !cd /content/HippoRAG; python src/named_entity_extraction_parallel.py --dataset $DATA --llm $LLM_API --model_name $LLM_MODEL  # NER for queries

# # Creating Contriever Graph
# !cd /content/HippoRAG; python src/create_graph.py --dataset $DATA --model_name $HF_RETRIEVER --extraction_model $LLM_MODEL --threshold $SYNONYM_THRESH --extraction_type $extraction_type --cosine_sim_edges

# # Getting Nearest Neighbor Files
# %env CUDA_VISIBLE_DEVICES=0
# !cd /content/HippoRAG; python src/RetrievalModule.py --retriever_name $HF_RETRIEVER --string_filename output/query_to_kb.tsv
# !cd /content/HippoRAG; python src/RetrievalModule.py --retriever_name $HF_RETRIEVER --string_filename output/kb_to_kb.tsv
# !cd /content/HippoRAG; python src/RetrievalModule.py --retriever_name $HF_RETRIEVER --string_filename output/rel_kb_to_kb.tsv

# !cd /content/HippoRAG; python src/create_graph.py --dataset $DATA --model_name $HF_RETRIEVER --extraction_model $LLM_MODEL --threshold $SYNONYM_THRESH --create_graph --extraction_type $extraction_type --cosine_sim_edges
